In [29]:
# pip install kalshi-python

You should consider upgrading via the 'c:\Users\arune\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python
from pprint import pprint

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
from kalshi_python.models import *
from datetime import datetime

In [2]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

# prod_email = "" # change these to be your personal credentials
# prod_password = "" # (for extra security, we recommend using a config file)

demo_email = "aklal@bu.edu" # change these to be your personal credentials
demo_password = "Sachin@007" # (for extra security, we recommend using a config file)

# for prod
# prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [3]:
config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email='aklal@bu.edu',
    password='Sachin@007',
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
print(exchangeStatus)

Exchange status response: 
{'exchange_active': True, 'trading_active': True}


In [4]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('predicted_temperature_max.csv')
print(df)
# # Get the predicted temperature for Austin
# predicted_temp_for_austin = df.loc[df['Location'] == 'Austin', 'predicted_tempmax'].values[0]
# print(f"Predicted Temperature for Austin: {predicted_temp_for_austin}")


   Location  predicted_tempmax
0  New York               49.0
1   Chicago               51.0
2    Austin               75.0
3     Miami               84.0


In [5]:
def determine_market_ticker_from_response(eventResponse, predicted_max_temp):
    for market in eventResponse.markets:
        subtitle = market.subtitle  # Adjusted based on your object structure
        if 'or below' in subtitle:
            max_temp = int(subtitle.split('°')[0])
            if predicted_max_temp <= max_temp:
                return market.ticker.split('-')[-1]
        elif 'or above' in subtitle:
            min_temp = int(subtitle.split('°')[0]) + 1
            if predicted_max_temp >= min_temp:
                return market.ticker.split('-')[-1]
        else:
            # Handling ranges correctly by cleaning the string of non-numeric before conversion
            temp_range = [int(temp.split('°')[0]) for temp in subtitle.split(' to ')]
            if temp_range[0] <= predicted_max_temp <= temp_range[1]:
                return market.ticker.split('-')[-1]
    return None


In [11]:
seriesTickers = {
    'New York': 'HIGHNY',
    'Chicago': 'HIGHCHI',
    'Austin': 'HIGHAUS',
    'Miami': 'HIGHMIA'
}
today = datetime.today()

# print(f"Processing cities: {df['Location'].tolist()}")

# for index, row in df.iterrows():
#     city = row['Location']
#     print(f"Processing {city}...")

# # Format the date as 'YYMMDD'
date_str = today.strftime('%y%b%d').upper()
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]
    

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)  # Replace with actual API call
    
    # Parse the event response to find matching market ticker
    # This code block assumes a function to parse and match the market ticker based on predicted temperature
    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    # Now you have the matching market ticker, you can retrieve the order book
    marketTicker = f'{eventTicker}-{matching_market_ticker}'
    orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Replace with actual API call
    
    print(f'\nOrderbook for market: {marketTicker}')
    # Assuming pprint is imported or defined
    pprint(orderbookResponse)
# print(dir(orderbookResponse))



Orderbook for market: HIGHNY-24APR03-B48.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHCHI-24APR03-T42
{'orderbook': {'no': [], 'yes': [[25, 50], [50, 20], [99, 1]]}}

Orderbook for market: HIGHAUS-24APR03-B74.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHMIA-24APR03-B84.5
{'orderbook': {'no': [], 'yes': [[30, 1000], [99, 1]]}}


In [ ]:
yes_offers = orderbookResponse.orderbook.yes
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']


In [16]:
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)

    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    if matching_market_ticker:
        marketTicker = f'{eventTicker}-{matching_market_ticker}'
        orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
        
        # Convert the orderbookResponse to a dictionary to access 'yes' and 'no' offers
        orderbook_dict = orderbookResponse.to_dict()
        yes_offers = orderbook_dict['orderbook']['yes'] if 'yes' in orderbook_dict['orderbook'] else []
        
        if yes_offers:
            best_yes_offer = min([offer[0] for offer in yes_offers])
            
            if exchangeStatus.trading_active:
                orderUuid = str(uuid.uuid4())
                orderResponse = kalshi_api.create_order(CreateOrderRequest(       
                    ticker=marketTicker,
                    action='buy',
                    type='limit',
                    yes_price=best_yes_offer - 1,  # Adjust based on your strategy
                    count=1000,  # And your desired amount
                    client_order_id=orderUuid,
                    side='yes',
                ))
                print(f'\nOrder submitted for {city}: ')
                pprint(orderResponse)
        else:
            print(f'\nNo "yes" offers available for {city}, no orders will be placed.')
    else:
        print(f'No matching market ticker found for {city} based on the predicted temperature.')
 


Order submitted for New York: 
{'order': {'action': 'buy',
           'client_order_id': '473a0902-560a-4bf6-972c-6983d1903f62',
           'created_time': '2024-04-03T15:05:17.307775Z',
           'expiration_time': None,
           'no_price': 2,
           'order_id': '60575f25-f4be-4ef1-b441-b66b12eafd5c',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHNY-24APR03-B48.5',
           'type': 'limit',
           'user_id': '8eb92856-b0cd-4c17-bc68-637fcde6ecfe',
           'yes_price': 98}}

Order submitted for Chicago: 
{'order': {'action': 'buy',
           'client_order_id': 'd5a4dc45-977a-48df-8344-7875518ceafa',
           'created_time': '2024-04-03T15:05:17.713731Z',
           'expiration_time': None,
           'no_price': 76,
           'order_id': '6d6ecf7d-c09e-429b-85e4-e08d4c86631d',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHCHI-24APR03-T42',
           'type': 'limit',
           'user_id': 

In [ ]:
# if matching_market_ticker:
#     marketTicker = f'{eventTicker}-{matching_market_ticker}'
#         # Assuming you have a function or method to fetch the current exchange status
#     if exchangeStatus.trading_active:  # Make sure this checks the current, actual trading status
#         orderUuid = str(uuid.uuid4())
#         orderResponse = kalshi_api.create_order(CreateOrderRequest(       
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,  # Adjust the price according to your strategy
#         count=1000,  # And the amount of contracts you want to buy
#         client_order_id=orderUuid,
#         side='yes',
#         ))
#         print(f'\nOrder submitted for {city}: ')
#         pprint(orderResponse)
#     else:
#         print(f'\nThe exchange is not trading active for {city}, no orders will be sent right now.')
# else:
#     print(f'No matching market ticker found for {city} based on the predicted temperature.')


In [15]:
# for index, row in df.iterrows():
#     # city = row['Location']
#     # predicted_max_temp = row['predicted_tempmax']
#     # seriesTicker = seriesTickers[city]

#     # eventTicker = f'{seriesTicker}-{date_str}'
#     # eventResponse = kalshi_api.get_event(eventTicker)  # Ensure this fetches the event data correctly
    
#     # matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
#     if matching_market_ticker:
#     #     marketTicker = f'{eventTicker}-{matching_market_ticker}'
#     #     orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Fetch the order book
        
#     #     print(f'\nOrderbook for market: {marketTicker}')
#     #     pprint(orderbookResponse)
        
#         # Example strategy: Place an order only if there's at least one 'yes' offer
#         if orderbookResponse['orderbook']['yes']:
#             # Assuming you analyze the order book to decide your order details
#             # best_yes_offer = min(orderbookResponse['orderbook']['yes'])[0]  # Example strategy
            
#             if exchangeStatus.trading_active:  # Confirm trading status
#                 orderUuid = str(uuid.uuid4())
#                 orderResponse = kalshi_api.create_order(CreateOrderRequest(       
#                     ticker=marketTicker,
#                     action='buy',
#                     type='limit',
#                     yes_price=30,
#                     # best_yes_offer - 1,  # Example strategy: bid just below the best 'yes' offer
#                     count=1000,
#                     client_order_id=orderUuid,
#                     side='yes',
#                 ))
#                 print(f'\nOrder submitted for {city}: ')
#                 pprint(orderResponse)
#             else:
#                 print(f'\nThe exchange is not trading active for {city}, no orders will be sent right now.')
#         else:
#             print(f'\nNo yes offers available for {city}, no orders will be placed.')
#     else:
#         print(f'No matching market ticker found for {city} based on the predicted temperature.')


In [17]:
# # Fetch the order book
# orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)

# # Assuming direct attribute access for simplicity
# yes_offers = orderbookResponse.orderbook.yes if orderbookResponse.orderbook.yes else []

# if yes_offers:
#     # Example: Choose a strategy based on the offers
#     best_yes_offer = min([offer[0] for offer in yes_offers])
    
#     if exchangeStatus.trading_active:
#         orderUuid = str(uuid.uuid4())
#         orderResponse = kalshi_api.create_order(CreateOrderRequest(
#             ticker=marketTicker,
#             action='buy',
#             type='limit',
#             yes_price=40,
#             # best_yes_offer - 1,  # Simplified strategy
#             count=1000,
#             client_order_id=orderUuid,
#             side='yes',
#         ))
#         print(f'\nOrder submitted for {city}: ')
#         pprint(orderResponse)
# else:
#     print(f'\nNo "yes" offers available for {city}, no orders will be placed.')


In [9]:
# seriesTicker='HIGHNY'
# seriesResponse = kalshi_api.get_series(seriesTicker)
# print('\nSeries: ' + seriesTicker)
# pprint(seriesResponse)
# # from datetime import datetime
# # act_balance = exchange_client.get_balance
# # tickers = {"nyc":'HIGHNY',"austin":'HIGHAUS',"chicago":'HIGHCHI',"miami":'HIGHMIA'}
# # today = datetime.now()
# # formatted_date = today.strftime("%y%b%d").upper()




In [10]:
# today = datetime.today()

# # # Format the date as 'YYMMDD'
# date_str = today.strftime('%y%b%d').upper()
# # Combine seriesTicker and formatted date to create the eventTicker
# eventTicker = f'{seriesTicker}-{date_str}'
# # print(eventTicker)
# # eventTicker = 'HIGHNY-24APR02'
# eventResponse = kalshi_api.get_event(eventTicker)
# print('\nEvent: ' + seriesTicker)
# # pprint(eventResponse)
# for market in eventResponse.markets:
#     print(f"Ticker: {market.ticker}, Subtitle: {market.subtitle}")

In [11]:
# # Gets the orderbook for a specific market.
# marketTicker = f'{eventTicker}-{'Specify your order here'}'
# orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
# print('\nOrderbook for market: ' + marketTicker)
# pprint(orderbookResponse)

In [12]:
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=20,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')

In [13]:
# #Chichago
# seriesTicker='HIGHCHI'
# seriesResponse = kalshi_api.get_series(seriesTicker)
# print('\nSeries: ' + seriesTicker)
# pprint(seriesResponse)

In [14]:
# today = datetime.today()

# # Format the date as 'DDMMMYY'
# date_str = today.strftime('%y%b%d').upper()
# # Combine seriesTicker and formatted date to create the eventTicker
# eventTicker = f'{seriesTicker}-{date_str}'
# # eventTicker = 'HIGHCHI-24APR02'
# eventResponse = kalshi_api.get_event(eventTicker)
# print('\nEvent: ' + seriesTicker)
# # pprint(eventResponse)
# for market in eventResponse.markets:
#     print(f"Ticker: {market.ticker}, Subtitle: {market.subtitle}")

In [16]:
# # Gets the orderbook for a specific market.
# marketTicker = f'{eventTicker}-{'Specify your order here'}'
# orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
# print('\nOrderbook for market: ' + marketTicker)
# pprint(orderbookResponse)

In [15]:
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')

In [17]:
# #Miami
# seriesTicker='HIGHMIA'
# seriesResponse = kalshi_api.get_series(seriesTicker)
# print('\nSeries: ' + seriesTicker)
# pprint(seriesResponse)

In [18]:
# today = datetime.today()

# # Format the date as 'DDMMMYY'
# date_str = today.strftime('%y%b%d').upper()
# # Combine seriesTicker and formatted date to create the eventTicker
# eventTicker = f'{seriesTicker}-{date_str}'

# # eventTicker = 'HIGHMIA-24APR02'
# eventResponse = kalshi_api.get_event(eventTicker)
# print('\nEvent: ' + seriesTicker)

# # pprint(eventResponse)
# for market in eventResponse.markets:
#     print(f"Ticker: {market.ticker}, Subtitle: {market.subtitle}")



In [21]:
# def determine_market_ticker_from_response(eventResponse, predicted_max_temp):
#     for market in eventResponse.markets:
#         subtitle = market.subtitle  # Adjusted based on your object structure
#         if 'or below' in subtitle:
#             max_temp = int(subtitle.split('°')[0])
#             if predicted_max_temp <= max_temp:
#                 return market.ticker.split('-')[-1]
#         elif 'or above' in subtitle:
#             min_temp = int(subtitle.split('°')[0]) + 1
#             if predicted_max_temp >= min_temp:
#                 return market.ticker.split('-')[-1]
#         else:
#             # Handling ranges correctly by cleaning the string of non-numeric before conversion
#             temp_range = [int(temp.split('°')[0]) for temp in subtitle.split(' to ')]
#             if temp_range[0] <= predicted_max_temp <= temp_range[1]:
#                 return market.ticker.split('-')[-1]
#     return None


In [22]:
# seriesTickers = {
#     'New York': 'HIGHNY',
#     'Chicago': 'HIGHCHI',
#     'Austin': 'HIGHAUS',
#     'Miami': 'HIGHMIA'
# }
# for index, row in df.iterrows():
#     city = row['Location']
#     predicted_max_temp = row['predicted_tempmax']
#     seriesTicker = seriesTickers[city]

#     eventTicker = f'{seriesTicker}-{date_str}'
#     eventResponse = kalshi_api.get_event(eventTicker)  # Replace with actual API call
    
#     # Parse the event response to find matching market ticker
#     # This code block assumes a function to parse and match the market ticker based on predicted temperature
#     matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
#     # Now you have the matching market ticker, you can retrieve the order book
#     marketTicker = f'{eventTicker}-{matching_market_ticker}'
#     orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Replace with actual API call
    
#     print(f'\nOrderbook for market: {marketTicker}')
#     # Assuming pprint is imported or defined
#     pprint(orderbookResponse)


In [23]:
# # Gets the orderbook for a specific market.
# def determine_market_ticker(predicted_temp, temp_ranges):
#     for (min_temp, max_temp), ticker in temp_ranges.items():
#         if min_temp is None and predicted_temp <= max_temp:
#             return ticker
#         elif max_temp is None and predicted_temp >= min_temp:
#             return ticker
#         elif min_temp is not None and max_temp is not None and min_temp <= predicted_temp <= max_temp:
#             return ticker
#     return "No matching market ticker found"
# temp_ranges = parse_market_data(eventResponse['markets'])
# # print(temp_ranges)
# Gets the orderbook for a specific market.
# marketTicker = f'{eventTicker}-{'order'}'
# orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
# print('\nOrderbook for market: ' + marketTicker)
# pprint(orderbookResponse)

In [24]:
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')

In [25]:
# #Austin
# seriesTicker='HIGHAUS'
# seriesResponse = kalshi_api.get_series(seriesTicker)
# print('\nSeries: ' + seriesTicker)
# pprint(seriesResponse)

In [26]:
# today = datetime.today()

# # Format the date as 'DDMMMYY'
# date_str = today.strftime('%y%b%d').upper()
# # Combine seriesTicker and formatted date to create the eventTicker
# eventTicker = f'{seriesTicker}-{date_str}'
# # eventTicker = 'HIGHAUS-24APR02'
# eventResponse = kalshi_api.get_event(eventTicker)
# print('\nEvent: ' + seriesTicker)
# # pprint(eventResponse)
# for market in eventResponse.markets:
#     print(f"Ticker: {market.ticker}, Subtitle: {market.subtitle}")

In [27]:
# # Gets the orderbook for a specific market.
# marketTicker = f'{eventTicker}-{}'
# orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
# print('\nOrderbook for market: ' + marketTicker)
# pprint(orderbookResponse)

In [28]:
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')
# if exchangeStatus.trading_active:
#     orderUuid = str(uuid.uuid4())
#     orderResponse = kalshi_api.create_order(CreateOrderRequest(
#         ticker=marketTicker,
#         action='buy',
#         type='limit',
#         yes_price=30,
#         count=1000,
#         client_order_id=orderUuid,
#         side='yes',
#     ))
#     print('\nOrder submitted: ')
#     pprint(orderResponse)
# else:
#     print('\nThe exchange is not trading active, no orders will be sent right now.')
